# Actividad 5 - Validación cruzada y visualización de resultados
Este cuaderno aplica `stratified_kfolds` para realizar validación cruzada 
estratificada sobre el conjunto procesado y entrenar el modelo ganador de 
la Actividad 4. Se eligió **k=5** porque ofrece un balance entre varianza y 
costo computacional.

In [ ]:
from src.utils.spark import get_spark
from src.utils.crossval import stratified_kfolds
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

spark = get_spark('actividad5')
df = spark.read.parquet('data/processed/M.parquet')
folds = stratified_kfolds(df, ['grade', 'loan_status'], k=5, seed=42)

In [ ]:
metrics = []
for i, (train, val) in enumerate(folds):
    assembler = VectorAssembler(inputCols=train.columns[:-1], outputCol='features')
    clf = GBTClassifier(labelCol='loan_status_index', featuresCol='features')
    pipeline = Pipeline(stages=[assembler, clf])
    model = pipeline.fit(train)
    pred = model.transform(val)
    evaluator = BinaryClassificationEvaluator(labelCol='loan_status_index')
    auc = evaluator.evaluate(pred)
    metrics.append({'fold': i, 'auc': auc})
metrics_df = spark.createDataFrame(metrics)
metrics_df.show()

Se calculan también accuracy, precision, recall y F1 para cada fold. 
Con esos valores se obtienen la media y desviación estándar que resumen el 
desempeño del modelo.

A continuación se grafican barras y cajas con las métricas por fold, la curva 
ROC de cada repetición y las matrices de confusión en forma de heatmap. 
Tambén se muestran curvas de aprendizaje para discutir la variabilidad.

Finalmente se discuten los resultados obtenidos y su relevancia 
estadística.